## How to set up an optimisation that uses U-NSGA-III as the acquisition function

U-NSGA-III can be used for single-objective and multi-objective problems. Like the TS-EMO method, samples are drawn from the objective functions' distributions as opposed to using the mean of the distribution directly during inference in the algorithm.

The U-NSGA-III algorithm is implemented in the `NSGAImprovement` acquisition function in NEMO. The following are the steps for how this acquisition function works:
1. The acquisition function will firstly use the U-NSGA-III algorithm to identify potential samples on the pareto front. A sampler object is not used for this step because the algorithm will actively generate it's own samples *in situ*. 
2. The new potential pareto front points will be checked against the existing pareto front in the database. The method tries to find a combination of `NSGAImprovement.num_candidates` number of data points from new potential pareto front points that improvement the existing hypervolume.
3. If step 2 is successful, then these points will be the suggested candidates by the acquisition function.
4. Alternatively, if step 2 fails, then the `HighestUncertainty` acquisition function is run to identify `NSGAImprovement.num_candidates` number of sets of X-values that have the highest predictive uncertainty (standard deviation) in the objective functions. The identified high uncertainty candidates are suggested as experiments to run.

In [ ]:
# Import the variable, objectives, sampler, acquisition function, and the optimisation classes
from nemo_bo.opt.variables import ContinuousVariable, VariablesList
from nemo_bo.opt.objectives import RegressionObjective, ObjectivesList
from nemo_bo.acquisition_functions.nsga_improvement.nsga_improvement import (
    NSGAImprovement,
)
from nemo_bo.opt.optimisation import Optimisation

In [ ]:
# Create the variable objects
var1 = ContinuousVariable(name="variable1", lower_bound=0.0, upper_bound=100.0)
var2 = ContinuousVariable(name="variable2", lower_bound=0.0, upper_bound=100.0)
var3 = ContinuousVariable(name="variable3", lower_bound=0.0, upper_bound=100.0)
var_list = VariablesList([var1, var2, var3])

In [ ]:
# Create the objective objects
obj1 = RegressionObjective(
    name="objective1",
    obj_max_bool=True,
    lower_bound=0.0,
    upper_bound=100.0,
)
obj2 = RegressionObjective(
    name="objective2",
    obj_max_bool=False,
    lower_bound=0.0,
    upper_bound=100.0,
)
obj_list = ObjectivesList([obj1, obj2])

Setting the number of suggested candidates, `num_candidates` to be more than 4 is not recommended due to the risk of long run times. The method used to determine the hypervolume improvement in NEMO dramatically increases in run time as the number of new potential pareto front points increases when `num_candidates` are greater than 4.

In [ ]:
# Instantiate the NSGAImprovement class to use as the acquisition function
acq_func = NSGAImprovement(num_candidates=2)

In [ ]:
# Set up the optimisation instance
optimisation = Optimisation(var_list, obj_list, acq_func)

In [ ]:
# Start the optimisation using the convenient run function that will run for the specified number of iterations
# X and Y arrays represent a hypothetical initial dataset
optimisation_data = optimisation.run(X, Y, number_of_iterations=50)